# Skip-gram Word2Vec

In this notebook, I'll lead you through using PyTorch to implement the [Word2Vec algorithm](https://en.wikipedia.org/wiki/Word2vec) using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in NLP. This will come in handy when dealinw with things like machine translation.

## Readings

Here are the resources used to build this notebook
* A [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of Word2Vec from Chris McCormick
* [First Word2Vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for Word2Vec also from Mikolov et al.

---

## Word Embeddings

When you're dealing with words in text, you end up with tens of thousands of word classes to analyze; one for each word in a vocabulary. Trying to one-hot encode these words is massively inefficient because most values in a one-hot vector will be set to zero. So the matrix multiplication that happens in between a one-hot input vector and a first, hidden layer will result in mostly zero-valued hidden outputs.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/word2vec-embeddings/assets/one_hot_encoding.png">

To solve this problem and greatly increase the efficiency of our network, we use what are called **embeddings**. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are the embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding to the index of the "on" input unit.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/word2vec-embeddings/assets/lookup_matrix.png">

Instead of doing the matrix multplication, we use the weight matrix as a lookup table. We encode the words as integers, for example, "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/word2vec-embeddings/assets/tokenize_lookup.png">

There is nothing magical going on here. The embedding lookup table is just a weight matrix. THe embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.

---

## Word2Vec

The Word2Vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/word2vec-embeddings/assets/context_drink.png">

Words that show up in similar **contexts**, such as "coffee", "tea", and "water" will have vectors near each other. Different words will be further away from one another, and relationships can be represented by distance in vector space.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/word2vec-embeddings/assets/vector_distance.png">

There are two architectures for implementing Word2Vec:

>* CBOW (Continuous Bag-Of_words) and 
>* Skip-gram

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/word2vec-embeddings/assets/word2vec_architectures.png">

In this implementation, we'll be using the **skip-gram architecture** because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts

---

## Loading Data

We will load in data and place it in the `data` directory.

1. Load the [text8 dataset](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip); a file of cleaned up *Wikipedia article text* from Matt Mahoney
2. Place that data in the `data` home directory
3. Then you can extract it and delete the archive, zip file to save storage space

After following these steps, you should have one file in your data directory: `data/text8`

In [2]:
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip
!mkdir ./data 
!unzip text8.zip -d ./data

--2021-01-23 22:53:27--  https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.251.166
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.251.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M  29.4MB/s    in 1.0s    

2021-01-23 22:53:28 (29.4 MB/s) - ‘text8.zip’ saved [31344016/31344016]

Archive:  text8.zip
caution: filename not matched:  ./data


In [4]:
# read in the extracted text file
with open('data/text8') as f:
    text = f.read()

# print out the first 100 characters
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


## Pre-processing